## Neural Network for training CIFAR10 dataset

#### Going to use:

1. Tensorflow - Open source machine learning library (Offers APIs for developers to use the library to create deep learning models e.g. Keras)
2. Keras - High level neural network API which sits on top of Tensorflow. Developers interact directly with Keras in order to easily use Tensorflow.

Keras is an easy way for users to interact with the deep learning library in python code. It sits on top of libraries (e.g. Tensorflow, Theano) and provides users with understandable feedback enabling faster prototyping.

Reference:
https://github.com/keras-team/keras/blob/master/examples/cifar10_cnn.py


## Keras Models

1. Sequential Model (Fixed Structure - Based on inear stack of layers)
2. Functional Model (Flexible Structure - Ability to build arbitrary graphs of layers)


In [ ]:
# Importing libraries 
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os

# Initializing variables 
batch_size = 10
num_classes = 10
epochs = 1
num_predictions = 20
model_name = 'keras_cifar10_trained_model.h5'

In [ ]:
# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


## Training a sequential model

### A1) Add list of layer instances

1. Can use either input_dim or input_shape
model.add(Conv2D(32, (3, 3), padding='same', input_shape=x_train.shape[1:]))
OR
model.add(Dense(32, input_dim=784))

2. input_dim = First value (x) of <x_train.shape> (x,y)
3. input_shape = Both values (x and y) of <I_train.shape>

### A2) Compile model to configure training process
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['mae'])

1. loss
2. optimizer 
3. metrics 

### A3) Fit the model 

model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)

### A4) Evaluate the model

scores = model.evaluate(x_test, y_test, verbose=1)

### A5) Predict the values based on the training values

prediction = model.predict(y_test, batch_size=None, verbose=0, steps=None, callbacks=None)


In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4)

# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])



### With data augmentation = False

50000/50000 [==============================] - 9s 174us/step - loss: 1.7892 - acc: 0.3463 - val_loss: 1.5155 - val_acc: 0.4561

Epoch 2/10

50000/50000 [==============================] - 8s 162us/step - loss: 1.4882 - acc: 0.4604 - val_loss: 1.3446 - val_acc: 0.5210

Epoch 3/10

50000/50000 [==============================] - 7s 149us/step - loss: 1.3348 - acc: 0.5219 - val_loss: 1.2337 - val_acc: 0.5644

Epoch 4/10

50000/50000 [==============================] - 8s 154us/step - loss: 1.2198 - acc: 0.5668 - val_loss: 1.1141 - val_acc: 0.6096

Epoch 5/10

50000/50000 [==============================] - 8s 159us/step - loss: 1.1304 - acc: 0.6011 - val_loss: 1.1012 - val_acc: 0.6014

Epoch 6/10

50000/50000 [==============================] - 8s 162us/step - loss: 1.0666 - acc: 0.6249 - val_loss: 1.0574 - val_acc: 0.6324

Epoch 7/10

50000/50000 [==============================] - 7s 149us/step - loss: 1.0114 - acc: 0.6440 - val_loss: 0.9347 - val_acc: 0.6697

Epoch 8/10

50000/50000 [==============================] - 7s 149us/step - loss: 0.9695 - acc: 0.6603 - val_loss: 0.9691 - val_acc: 0.6579

Epoch 9/10

50000/50000 [==============================] - 8s 154us/step - loss: 0.9343 - acc: 0.6723 - val_loss: 0.8801 - val_acc: 0.6933

Epoch 10/10

50000/50000 [==============================] - 8s 165us/step - loss: 0.9009 - acc: 0.6846 - val_loss: 0.8746 - val_acc: 0.6910

10000/10000 [==============================] - 0s 47us/step


Test loss: 0.874568017578125

Test accuracy: 0.691

### With data augmentation = True

10000/10000 [==============================] - 75s 8ms/step - loss: 1.4662 - acc: 0.4706 - val_loss: 1.1131 - val_acc: 0.5975

Epoch 2/10

10000/10000 [==============================] - 74s 7ms/step - loss: 1.1070 - acc: 0.6077 - val_loss: 0.8593 - val_acc: 0.6979

Epoch 3/10

10000/10000 [==============================] - 74s 7ms/step - loss: 0.9656 - acc: 0.6621 - val_loss: 0.8565 - val_acc: 0.7050

Epoch 4/10

10000/10000 [==============================] - 74s 7ms/step - loss: 0.8877 - acc: 0.6924 - val_loss: 0.7928 - val_acc: 0.7311

Epoch 5/10

10000/10000 [==============================] - 74s 7ms/step - loss: 0.8451 - acc: 0.7090 - val_loss: 0.7708 - val_acc: 0.7344

Epoch 6/10

10000/10000 [==============================] - 74s 7ms/step - loss: 0.8248 - acc: 0.7184 - val_loss: 0.7490 - val_acc: 0.7505

Epoch 7/10

10000/10000 [==============================] - 74s 7ms/step - loss: 0.8075 - acc: 0.7255 - val_loss: 0.7116 - val_acc: 0.7585

Epoch 8/10

10000/10000 [==============================] - 74s 7ms/step - loss: 0.7989 - acc: 0.7301 - val_loss: 0.6993 - val_acc: 0.7637

Epoch 9/10

10000/10000 [==============================] - 74s 7ms/step - loss: 0.7894 - acc: 0.7346 - val_loss: 0.6806 - val_acc: 0.7781

Epoch 10/10

10000/10000 [==============================] - 74s 7ms/step - loss: 0.7873 - acc: 0.7360 - val_loss: 0.6979 - val_acc: 0.7691

10000/10000 [==============================] - 1s 52us/step


Test loss: 0.6979419784545898

Test accuracy: 0.7691